## 1. Imports & Config

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
os.chdir("..")


In [2]:
import yaml
import pandas as pd
from pathlib import Path

def load_config(config_path: str = "config.yaml") -> dict:
    """
    Lit le fichier YAML de configuration et renvoie un dict.
    """
    with open(config_path, "r") as f:
        cfg = yaml.safe_load(f)
    # Remplace ${VARNAME} par la vraie variable d'env si nécessaire
    for section in cfg.values():
        if isinstance(section, dict):
            for k, v in section.items():
                if isinstance(v, str) and v.startswith("${") and v.endswith("}"):
                    var = v[2:-1]
                    section[k] = os.getenv(var)
    return cfg

# Charger config
cfg = load_config("config.yaml")
raw_dir = Path(cfg["paths"]["data"]["raw"])
ext_dir = Path(cfg["paths"]["data"]["external"])
inter_dir = Path(cfg["paths"]["data"]["interim"])
inter_dir.mkdir(parents=True, exist_ok=True)

# Liste des zones
zones = list(cfg["geo_zones"].values())


## 2. Loader Functions


In [3]:
parent_region = {
    "Gran_canaria":             "Canarias",
    "Lanzarote_Fuerteventura":  "Canarias",
    "Tenerife":                 "Canarias",
    "La_Palma":                 "Canarias",
    "La_Gomera":                "Canarias",
    "El_Hierro":                "Canarias",
    "Mallorca_Menorca":         "Baleares",
    "Ibiza_Formentera":         "Baleares",
    # Les autres sont directes
    "Peninsule_Iberique":       "Peninsule_Iberique",
    "Canarias":                 "Canarias",
    "Baleares":                 "Baleares",
    "Ceuta":                    "Ceuta",
    "Melilla":                  "Melilla",
    "nacional":                 "España"
}


### 2.1 Charger la demande électrique
def load_demand(zone):
    df = pd.read_csv(raw_dir/f"{zone}_hourly.csv",
                     parse_dates=["datetime"],
                     index_col="datetime")
    df = df.resample("h").sum()
    df = df.rename(columns={"value": "demand"})
    return df

### 2.2 Charger le prix de l'électricité
def load_pvpc(zone: str):
    """
    Charge le fichier PVPC pour `zone`. 
    Si absent, cherche dans sa région parente.
    """
    filename = f"{zone}_pvpc_hourly.csv"
    path = ext_dir/"prices"/filename
    if not path.exists():
        parent = parent_region.get(zone)
        if parent and parent != zone:
            print(f"⚠️  Pas de fichier pour {zone}, utilisation de {parent} à la place")
            path = ext_dir/"prices"/f"{parent}_pvpc_hourly.csv"
        else:
            raise FileNotFoundError(f"Aucun fichier PVPC pour {zone} ni parent")
    df = pd.read_csv(path, parse_dates=["datetime"], index_col="datetime")
    df = df.rename(columns={"value": "pvpc"})
    return df

### 2.3 les données météo
def load_weather(zone):
    if zone == "nacional":
        # On charge les données de la péninsule ibérique
        zone = "Peninsule_Iberique"
    df = pd.read_csv(ext_dir/"weather"/f"{zone}_weather_hourly.csv",
                     parse_dates=["datetime"],
                     index_col="datetime")
    return df  # colonnes: temperature_2m, relative_humidity_2m, etc.

### 2.4 les jours fériés
def load_holidays(zone: str):
    """
    Charge le fichier des jours fériés pour `zone`.
    Si absent, cherche dans sa région parente.
    """
    if zone == "nacional":
        zone = "spain"
    filename = f"{zone}_holidays.csv"
    path = ext_dir/"holidays"/filename
    if not path.exists():
        parent = parent_region.get(zone)
        if parent and parent != zone:
            print(f"⚠️  Pas de fichier pour {zone}, utilisation de {parent} à la place")
            path = ext_dir/"holidays"/f"{parent}_holidays.csv"
        else:
            raise FileNotFoundError(f"Aucun fichier de jours fériés pour {zone} ni parent")
    df = pd.read_csv(path, parse_dates=["date"])
    df["is_holiday"] = 1
    df = df.set_index("date")[["is_holiday"]]
    # Localiser l'index en UTC pour matcher df.date
    df.index = df.index.tz_localize("UTC")
    return df




## 3. Prétraitement horaire et fusion


In [ ]:
def preprocess_zone(zone):
    # Charger sources
    df_dem = load_demand(zone)      # index datetime64[ns]
    df_pv  = load_pvpc(zone)        # index datetime64[ns]
    df_wth = load_weather(zone)     # index datetime64[ns]
    df_hol = load_holidays(zone)    # index date (DatetimeIndex of dates)

    # Fusion horaire
    df = df_dem.join(df_pv, how="left") \
               .join(df_wth, how="left")

    # Créer une colonne 'date' au format datetime64[ns] (00:00)
    df["date"] = df.index.normalize()

    # Merge sur date, en laissant df_hol index en DatetimeIndex
    df = df.merge(
        df_hol,
        left_on="date",
        right_index=True,
        how="left"
    )

    
    df["is_holiday"] = df["is_holiday"].shift(-2, freq="h") 
    # Remplir les NaN, supprimer la colonne intermédiaire
    df["is_holiday"] = df["is_holiday"].fillna(0).astype(int)
    df = df.drop(columns="date")

    return df


## 4. Features temporelles


In [5]:
import numpy as np

def add_time_features(df):
    idx = df.index
    df["hour"]      = idx.hour
    df["dayofweek"] = idx.dayofweek
    df["is_weekend"]= (idx.dayofweek >= 5).astype(int)
    df["month"]     = idx.month
    df["dayofyear"] = idx.dayofyear
    # cyclic encoding
    df["hour_sin"]  = np.sin(2*np.pi*df["hour"]/24)
    df["hour_cos"]  = np.cos(2*np.pi*df["hour"]/24)
    return df

## 5. Lags & Rolling

In [6]:
def add_lag_rolling(df):
    df["lag_1h"]  = df["demand"].shift(1)
    df["lag_24h"] = df["demand"].shift(24)
    df["lag_168h"]= df["demand"].shift(168)
    df["rmean_7d"]= df["demand"].rolling(24*7).mean()
    df["rstd_7d"] = df["demand"].rolling(24*7).std()
    return df


## 6. Handling Missing

In [7]:
def fill_missing(df):
    # Forward fill, puis backward for leading NaNs
    df = df.ffill().bfill()
    return df


## 7. Pipeline complet & sauvegarde


In [21]:
import numpy as np

for zone in zones:
    print(f"Processing {zone}...")
    df_z = preprocess_zone(zone)
    df_z = add_time_features(df_z)
    df_z = add_lag_rolling(df_z)
    df_z = fill_missing(df_z)
    # Sauvegarde
    out_path = inter_dir/f"{zone}_interim_hourly.pkl"
    df_z.to_pickle(out_path)
    print(f"  → saved to {out_path.name}")

    df_daily = df_z.resample(cfg["processing"]["aggregate_freq"]).agg({
        "demand": "sum",            # ou mean si vous préférez
        "pvpc":   "mean",
        # météo :
        "temperature_2m":      "mean",
        "relative_humidity_2m":"mean",
        "wind_speed_10m":      "mean",
        "shortwave_radiation": "sum",
        "precipitation":       "sum",
        # indicateurs temporels :
        "is_holiday": lambda x: x.mode()[0] if not x.mode().empty else 0,
        "is_weekend": "max"
    })

    # puis lags journaliers, features date (dayofweek, month…), etc.
    df_daily["lag_1d"]   = df_daily["demand"].shift(1)
    df_daily["lag_7d"]   = df_daily["demand"].shift(7)
    df_daily["lag_30d"]  = df_daily["demand"].shift(30)
    df_daily["lag_365d"] = df_daily["demand"].shift(365)
    df_daily["rmean_7d"] = df_daily["demand"].rolling(7).mean()
    df_daily["rstd_7d"]  = df_daily["demand"].rolling(7).std()

    # Sauvegarde
    df_daily.to_pickle(inter_dir/f"{zone}_interim_daily.pkl")


Processing Peninsule_Iberique...
  → saved to Peninsule_Iberique_interim_hourly.pkl
Processing Baleares...
  → saved to Baleares_interim_hourly.pkl
Processing Canarias...
  → saved to Canarias_interim_hourly.pkl
Processing Gran_canaria...
⚠️  Pas de fichier pour Gran_canaria, utilisation de Canarias à la place
⚠️  Pas de fichier pour Gran_canaria, utilisation de Canarias à la place
  → saved to Gran_canaria_interim_hourly.pkl
Processing Ceuta...
  → saved to Ceuta_interim_hourly.pkl
Processing Melilla...
  → saved to Melilla_interim_hourly.pkl
Processing Lanzarote_Fuerteventura...
⚠️  Pas de fichier pour Lanzarote_Fuerteventura, utilisation de Canarias à la place
⚠️  Pas de fichier pour Lanzarote_Fuerteventura, utilisation de Canarias à la place
  → saved to Lanzarote_Fuerteventura_interim_hourly.pkl
Processing Tenerife...
⚠️  Pas de fichier pour Tenerife, utilisation de Canarias à la place
⚠️  Pas de fichier pour Tenerife, utilisation de Canarias à la place
  → saved to Tenerife_inter